# Generate Prompt Candidates

In [54]:
#!pip3 install pydantic
#!pip3 install instructor


In [55]:
# imports
from pydantic import BaseModel
from openai import OpenAI
from typing import List
import instructor

In [82]:
# general variables
number_of_prompts = 5
model = "gpt-4o"
temp_prompt_generation = 0.2
# temperature for extraction and creating modell tasks
temp_task = 0

# paths process descs
path_nebentaetigkeiten = "assets/process_desc_nebentaetigkeiten.txt"
path_bedarf = "assets/process_desc_bedarfsermittlung.txt"
path_defbriefing = "assets/process_desc_debriefing.txt"

# msgs for generating prompts
sys_msg_generating_prompts = "Du bist ein prompt engineering Experte, der spezialisiert ist auf die Erstellung passender Prompts für eine gegebene Aufgabe."

In [76]:
# task specific variables ROLE EXTRACTION

task_role_extraction = "Extrahiere alle Rollen aus der folgenden Prozessbeschreibung und gib diese als Liste zurück:"
sys_msg_role_extraction = "Du bist ein Prozessmanager, bekommst eine Prozessbeschreibung und extrahierst alle Rollen aus dieser, im vom Benutzer definierten Format."

sys_msg_eval_roles = '''Du bist eine Evaluierungssystem, spezialisiert darin extrahierte Rollen aus Prozessbeschreibungen zu bewerten.
Du gibst immer eine Punktzahl zwischen 0 und 1 zurück.
Diese basiert darauf wie viele der extrahierten Rollen in den gegebenen Rollen vorkommen.
'''

In [77]:
# task specific variables ACTIVITY EXTRACTION

task_activity_extraction = "Extrahiere alle Aktivitäten aus der folgenden Prozessbeschreibung und gib diese als Liste zurück:"
sys_msg_activity_extraction = "Du bist ein Prozessmanager, bekommst eine Prozessbeschreibung und extrahierst alle Aktivitäten aus dieser, im vom Benutzer definierten Format."

sys_msg_eval_activities = '''Du bist eine Evaluierungssystem, spezialisiert darin extrahierte Aktivitäten aus Prozessbeschreibungen zu bewerten.
Du gibst immer eine Punktzahl zwischen 0 und 1 zurück.
Diese basiert darauf wie viele der extrahierten Aktivitäten in den gegebenen Aktivitäten vorkommen.
Extrahierte Aktivitäten dürfen anders als die gegebenen Aktivitäten geschrieben sein, jedoch muss der Kern der Aktivität der gleiche sein.
'''

In [73]:
class PromptCandidates(BaseModel):
    prompt_candidates: List[str]
    
client_structured = instructor.from_openai(OpenAI())
client = OpenAI()    
    
def generate_prompt_candidates(prompt: str):
    response = client_structured.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": sys_msg_generating_prompts},
                  {"role": "user", "content": prompt}],
        response_model=PromptCandidates,
        temperature=temp_prompt_generation)
    
    return response

## Creating Prompt Candidates for role extraction

In [74]:
prompt_generate_candidates = f"erstelle eine Liste mit {number_of_prompts} prompts für die folgende Aufgabe:\n\n{task_role_extraction}"

response_prompt_generation = generate_prompt_candidates(prompt_generate_candidates)
response_prompt_generation

PromptCandidates(prompt_candidates=['Lies die Prozessbeschreibung sorgfältig durch und liste alle erwähnten Rollen auf.', 'Identifiziere und extrahiere alle Rollen, die in der Prozessbeschreibung genannt werden, und gib sie als Liste zurück.', 'Erstelle eine Liste aller Rollen, die in der Prozessbeschreibung vorkommen.', 'Analysiere die Prozessbeschreibung und extrahiere alle Rollen, die darin erwähnt werden.', 'Finde alle Rollen in der Prozessbeschreibung und gib sie in Listenform zurück.'])

## Output of generated prompts

In [59]:
prompt_candidates = response_prompt_generation.prompt_candidates
for prompt in prompt_candidates:
    print(prompt)

Lies die Prozessbeschreibung sorgfältig durch und notiere alle erwähnten Rollen in einer Liste.
Identifiziere alle Rollen, die in der Prozessbeschreibung genannt werden, und liste sie auf.
Extrahiere jede Rolle aus der Prozessbeschreibung und gib sie als Liste zurück.
Analysiere die Prozessbeschreibung und erstelle eine Liste aller darin vorkommenden Rollen.
Durchsuche die Prozessbeschreibung nach Rollen und präsentiere sie in Listenform.


## Scoring of extraction prompts (Roles, Activities)

In [86]:
from pydantic import Field

class SummaryScoreRoles(BaseModel):
    overall_score: float = Field(description="Bewertet die Anzahl der extrahierten Rollen die in den gegebenen Rollen vorkommen")
    
class SummaryScoreActivities(BaseModel):
    overall_score: float = Field(description="Bewertet die Anzahl der extrahierten Aktivitäten die in den gegebenen Aktivitäten vorkommen")
    

def eval_output_roles(given_objects: str, roles_extracted: str):
    scoring_prompt = f"Folgende Rollen sind gegeben: {given_objects} \n\
        Bewerte, mit Blick auf die gegebenen Rollen, die Qualität der generierten Ausgabe: \n\
            {roles_extracted}."
            
    response_eval = client_structured.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_msg_eval_roles},
            {"role": "user", "content": scoring_prompt}
        ],
        response_model=SummaryScoreRoles)
    
    return response_eval

def eval_output_activities(given_objects: str, activities_extracted: str):
    scoring_prompt = f"Folgende Aktivitäten sind gegeben: {given_objects} \n\
        Bewerte, mit Blick auf die gegebenen Aktivitäten, die Qualität der generierten Ausgabe: \n\
            {activities_extracted}."
            
    response_eval = client_structured.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_msg_eval_activities},
            {"role": "user", "content": scoring_prompt}
        ],
        response_model=SummaryScoreActivities)
    
    return response_eval


## Preparing extraction (roles, activities)

In [89]:
def execute_extract_prompt(prompt_extraction: str, sys_msg_extraction: str):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_msg_extraction},
            {"role": "user", "content": prompt_extraction}
        ],      
        temperature=temp_task)
    
    return response.choices[0].message.content

def load_process_desc(path):
    file_process_desc = open(path, encoding="utf8")
    process_desc = file_process_desc.read()
    file_process_desc.close()
    
    return process_desc


In [62]:
def setup_prompt_extraction(prompt_candidate: str, process_desc: str):
    prompt__extraction = prompt_candidate + "\n" + "'''" + process_desc + "'''"
    return prompt__extraction

# Generating and evaluating prompts for role extraction

In [65]:
import pandas as pd

given_roles_nebentaetigkeiten = "Mitarbeiter, HR, Geschäftsführung"
given_roles_bedarf = "Geschäftsführung, HR-Committee, Staffing-Meeting, Kunden, Mitarbeiter, CEO, Tribe-Lead, FinCo"
given_roles_debriefing = "HR, Mitarbeiter, Abteilung, Geschäftsführung"

df = pd.DataFrame(columns=["Prompt", "Score", "Extracted Objects", "Input Process Desc", "Given Roles"])

process_objects = [[path_nebentaetigkeiten, given_roles_nebentaetigkeiten], [path_bedarf, given_roles_bedarf], [path_defbriefing, given_roles_debriefing]]


for object in process_objects:
    process_desc = load_process_desc(object[0])
    given_roles = object[1]
    
    for prompt in prompt_candidates:
        extraction_prompt = setup_prompt_extraction(prompt, process_desc)
        extracted_objects = execute_extract_prompt(extraction_prompt, sys_msg_role_extraction)
        eval_score = eval_output_roles(given_roles, extracted_objects)
        df.loc[len(df)] = [extraction_prompt, eval_score, extracted_objects, process_desc, given_roles]
    
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)

# All dataframes hereafter reflect these changes.
display(df)

,Prompt,Score,Extracted Objects,Input Process Desc,Given Roles
0,"Lies die Prozessbeschreibung sorgfältig durch und notiere alle erwähnten Rollen in einer Liste.\n'''Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden. \nUnterschriebenes Dokument einscannen und in der Personalakte ablegen. '''",overall_score=1.0,1. Mitarbeiter\n2. HR\n3. Geschäftsführung,"Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden. \nUnterschriebenes Dokument einscannen und in der Personalakte ablegen.","Mitarbeiter, HR, Geschäftsführung"
1,"Identifiziere alle Rollen, die in der Prozessbeschreibung genannt werden, und liste sie auf.\n'''Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden. \nUnterschriebenes Dokument einscannen und in der Personalakte ablegen. '''",overall_score=1.0,1. Mitarbeiter\n2. HR\n3. Geschäftsführung,"Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden. \nUnterschriebenes Dokument einscannen und in der Personalakte ablegen.","Mitarbeiter, HR, Geschäftsführung"
2,"Extrahiere jede Rolle aus der Prozessbeschr

## Generating Prompt Candidates for Activity Extraction

In [78]:
prompt_generate_candidates = f"erstelle eine Liste mit {number_of_prompts} prompts für die folgende Aufgabe:\n\n{task_activity_extraction}"

client_structured = instructor.from_openai(OpenAI())
client = OpenAI()

response_prompt_generation = generate_prompt_candidates(prompt_generate_candidates)
response_prompt_generation

PromptCandidates(prompt_candidates=['Bitte extrahiere alle Aktivitäten aus der untenstehenden Prozessbeschreibung und liste sie auf.', 'Identifiziere und liste alle Aktivitäten aus der folgenden Prozessbeschreibung auf.', 'Extrahiere die Aktivitäten aus der Prozessbeschreibung und gib sie als Liste zurück.', 'Lies die Prozessbeschreibung und extrahiere alle darin enthaltenen Aktivitäten als Liste.', 'Analysiere die Prozessbeschreibung und erstelle eine Liste aller Aktivitäten.'])

In [79]:
prompt_candidates = response_prompt_generation.prompt_candidates
for prompt in prompt_candidates:
    print(prompt)

Bitte extrahiere alle Aktivitäten aus der untenstehenden Prozessbeschreibung und liste sie auf.
Identifiziere und liste alle Aktivitäten aus der folgenden Prozessbeschreibung auf.
Extrahiere die Aktivitäten aus der Prozessbeschreibung und gib sie als Liste zurück.
Lies die Prozessbeschreibung und extrahiere alle darin enthaltenen Aktivitäten als Liste.
Analysiere die Prozessbeschreibung und erstelle eine Liste aller Aktivitäten.


In [ ]:
given_activities_nebentaetigkeiten = "Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben, Informationen einholen, Prüfen ob die Nebentätigkeit im Wettbewerb zu adorsys steht, Ablehnung der Nebentätigkeit, Dokument “Erlaubnis Nebentätigkeit” erstellen, Rücksprache mit dem Mitarbeiter oder der Geschäftsführung halten, Dokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden, Unterschriebenes Dokument einscannen und in der Personalakte ablegen"
given_activities_bedarf = "Prozessauslösung, Automatische Wiederholung des Prozesses, Bereitstellung von Bedarfssituationen, Detaillierung der Anforderungen im Staffing-Meeting, Ermittlung von internen Bedarfen, Freigabe des Stellenbedarfs, Veröffentlichung der Stellenausschreibung, Einholen von Informationen, Löschung der Ausschreibung nach erfolgreicher Einstellung"
given_activities_debriefing = "Debriefing-Gespräch führen, nach dem Grund der Kündigung fragen, Offene Themen klären, Projektübergaben, Kündigungsfrist, wann wäre letzter Arbeitstag, Besonderheiten, Hardware/Assets-Rückgabe (Was, wann) ,Feedback erfragen (freiwillig), Gespräch dokumentieren und ablegen"


df = pd.DataFrame(columns=["Prompt", "Score", "Extracted Objects", "Input Process Desc", "Given Roles"])

process_objects = [[path_nebentaetigkeiten, given_activities_nebentaetigkeiten], [path_bedarf, given_activities_bedarf], [path_defbriefing, given_activities_debriefing]]


for object in process_objects:
    process_desc = load_process_desc(object[0])
    given_activities = object[1]
    
    for prompt in prompt_candidates:
        extraction_prompt = setup_prompt_extraction(prompt, process_desc)
        extracted_objects = execute_extract_prompt(extraction_prompt, sys_msg_activity_extraction)
        eval_score = eval_output_activities(given_activities, extracted_objects)
        df.loc[len(df)] = [extraction_prompt, eval_score, extracted_objects, process_desc, given_activities]
    
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)

# All dataframes hereafter reflect these changes.
display(df)

## Generating Prompts for Process Model Creation

In [94]:
# Model Creation specific variables
task_model_creation = "Erstelle aus den gegebenen Rollen, Aktivitäten Du bist ein Prozessmanager. Dir werden Rollen, Aktivitäten und eine Prozessbeschreibung gegeben. Daraus erstellst du ein Draw.io konformes XML Dokument.und der Prozessbeschreibung ein Prozessmodell als XML Datei. Das XML muss Draw.io konform sein. Gib mir nur das Ergebnis in XML zurück. Es sollen alle Aktivitäten und Rollen verwendet werden."
sys_msg_model_creation = ""


In [97]:
prompt_generate_candidates = f"erstelle eine Liste mit {number_of_prompts} prompts für die unten genannte Aufgabe.\
    Aufgabe :\n\n{task_model_creation}"

client_structured = instructor.from_openai(OpenAI())
client = OpenAI()

response_prompt_generation = generate_prompt_candidates(prompt_generate_candidates)
response_prompt_generation

PromptCandidates(prompt_candidates=['Erstelle ein XML-Prozessmodell für Draw.io basierend auf den angegebenen Rollen, Aktivitäten und der Prozessbeschreibung. Verwende alle Informationen und liefere das Ergebnis ausschließlich als XML.', 'Generiere ein Draw.io-konformes XML-Prozessmodell, das alle bereitgestellten Rollen und Aktivitäten umfasst. Gib das XML-Dokument als Ergebnis zurück.', 'Nutze die gegebenen Rollen, Aktivitäten und die Prozessbeschreibung, um ein XML-Prozessmodell für Draw.io zu erstellen. Stelle sicher, dass alle Elemente integriert sind und liefere nur das XML.', 'Erstelle ein vollständiges Prozessmodell in XML-Format für Draw.io unter Verwendung aller angegebenen Rollen und Aktivitäten. Gib das XML als einziges Ergebnis zurück.', 'Verwende die bereitgestellten Rollen und Aktivitäten, um ein Draw.io XML-Prozessmodell zu erstellen. Stelle sicher, dass das XML alle erforderlichen Informationen enthält und liefere nur das XML zurück.'])

In [98]:
prompt_candidates = response_prompt_generation.prompt_candidates
for prompt in prompt_candidates:
    print(prompt)

Erstelle ein XML-Prozessmodell für Draw.io basierend auf den angegebenen Rollen, Aktivitäten und der Prozessbeschreibung. Verwende alle Informationen und liefere das Ergebnis ausschließlich als XML.
Generiere ein Draw.io-konformes XML-Prozessmodell, das alle bereitgestellten Rollen und Aktivitäten umfasst. Gib das XML-Dokument als Ergebnis zurück.
Nutze die gegebenen Rollen, Aktivitäten und die Prozessbeschreibung, um ein XML-Prozessmodell für Draw.io zu erstellen. Stelle sicher, dass alle Elemente integriert sind und liefere nur das XML.
Erstelle ein vollständiges Prozessmodell in XML-Format für Draw.io unter Verwendung aller angegebenen Rollen und Aktivitäten. Gib das XML als einziges Ergebnis zurück.
Verwende die bereitgestellten Rollen und Aktivitäten, um ein Draw.io XML-Prozessmodell zu erstellen. Stelle sicher, dass das XML alle erforderlichen Informationen enthält und liefere nur das XML zurück.


In [ ]:
#Zero Shot Prompting: "Erstelle ein Prozessmodell als XML Datei aus den gegebenen Rollen, Aktivitäten und der Prozessbeschreibung. Das XML muss Draw.io konform sein und alle Aktivitäten und Rollen verwenden. Gib mir nur das Ergebnis in XML zurück."
#Few Shot Prompting: "Hier sind einige Beispiele für Prozessmodelle in XML, die Draw.io konform sind: [Beispiel 1], [Beispiel 2]. Erstelle nun ein ähnliches Prozessmodell aus den gegebenen Rollen, Aktivitäten und der Prozessbeschreibung. Verwende alle Aktivitäten und Rollen und gib mir nur das Ergebnis in XML zurück."
#Chain of Thought: "Beginne mit der Identifikation aller Rollen und Aktivitäten. Ordne dann die Aktivitäten den entsprechenden Rollen zu. Erstelle basierend auf dieser Zuordnung ein Prozessmodell als XML Datei, das Draw.io konform ist. Stelle sicher, dass alle Aktivitäten und Rollen verwendet werden. Gib mir nur das Ergebnis in XML zurück."
#Knowledge Generation: "Nutze dein Wissen über Prozessmodellierung und XML-Strukturen, um aus den gegebenen Rollen, Aktivitäten und der Prozessbeschreibung ein Prozessmodell zu erstellen. Das XML muss Draw.io konform sein. Verwende alle Aktivitäten und Rollen und gib mir nur das Ergebnis in XML zurück."
#Kombination von Few Shot Prompting und Chain of Thought: "Hier sind einige Beispiele für Draw.io konforme Prozessmodelle in XML: [Beispiel 1], [Beispiel 2]. Identifiziere zunächst alle Rollen und Aktivitäten aus der Prozessbeschreibung. Ordne dann die Aktivitäten den entsprechenden Rollen zu und erstelle ein Prozessmodell als XML Datei. Stelle sicher, dass alle Aktivitäten und Rollen verwendet werden und gib mir nur das Ergebnis in XML zurück."

In [99]:
def setup_prompt_modell_creation(prompt_candidate: str, process_desc: str, input_roles: str, input_activities: str):
    prompt_modell_creation = prompt_candidate + "\n" + "'''Process Description: \n" + process_desc + " \n Given Roles:\n " + input_roles + "\n Given Activities:\n" + input_activities + "\n'''"
    return prompt_modell_creation

def execute_prompt_modell_creation(prompt_creation: str, sys_msg_creation: str):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_msg_creation},
            {"role": "user", "content": prompt_creation}
        ],      
        temperature=temp_task)
    
    return response.choices[0].message.content

In [101]:
#given_process_modell_nebentaetigkeiten?
#given_process_modell_bedarf?
#given_process_modell_debriefing?


df = pd.DataFrame(columns=["Prompt", "Score", "Extracted Model as draw.io file", "Input Process Desc", "Input Roles", "Input Activities"])

# Bedarf skipped cause of complicated process description-----[path_bedarf, given_activities_bedarf],
process_objects = [[path_nebentaetigkeiten, given_roles_nebentaetigkeiten, given_activities_nebentaetigkeiten], [path_defbriefing, given_roles_debriefing, given_activities_debriefing]]


for object in process_objects:
    process_desc = load_process_desc(object[0])
    input_roles = object[1]
    input_activities = object[2]
    
    for prompt in prompt_candidates:
        model_creation_prompt = setup_prompt_modell_creation(prompt, process_desc, input_roles, input_activities)
        model_created = execute_prompt_modell_creation(model_creation_prompt, sys_msg_model_creation)
        #eval_score = eval_output_activities(given_activities, extracted_objects)
        df.loc[len(df)] = [model_creation_prompt, "?", model_created, process_desc, input_roles, input_activities]
    
# All dataframes hereafter reflect these changes.
display(df)

,Prompt,Score,Extracted Model as draw.io file,Input Process Desc,Input Roles,Input Activities
0,"Erstelle ein XML-Prozessmodell für Draw.io basierend auf den angegebenen Rollen, Aktivitäten und der Prozessbeschreibung. Verwende alle Informationen und liefere das Ergebnis ausschließlich als XML.\n'''Process Description: \nMitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen un...",?,"```xml\n<mxfile host=""app.diagrams.net"">\n <diagram name=""Prozessmodell"" id=""prozessmodell"">\n <mxGraphModel dx=""1000"" dy=""1000"" grid=""1"" gridSize=""10"" guides=""1"" tooltips=""1"" connect=""1"" arrows=""1"" fold=""1"" page=""1"" pageScale=""1"" pageWidth=""827"" pageHeight=""1169"" math=""0"" shadow=""0"">\n <root>\n <mxCell id=""0"" />\n <mxCell id=""1"" parent=""0"" />\n \n <!-- Start Event -->\n <mxCell id=""start"" value=""Start"" style=""ellipse;whiteSpace=wrap;html=1;"" vertex=""1"" parent=""1"">\n <mxGeometry x=""100"" y=""100"" width=""40"" height=""40"" as=""geometry"" />\n </mxCell>\n \n <!-- Mitarbeiter informiert HR -->\n <mxCell id=""informieren"" value=""Mitarbeiter informiert HR"" style=""rounded=0;whiteSpace=wrap;html=1;"" vertex=""1"" parent=""1"">\n <mxGeometry x=""200"" y=""100"" width=""160"" height=""60"" as=""geometry"" />\n </mxCell>\n \n <!-- Informationen einholen -->\n <mxCell id=""informationen"" value=""Info...","Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/Woche (beginnend ab dem Eintritt bei adorsys) \n\nHR prüft, ob die Nebentätigkeit im Wettbewerb zu adorsys steht: \nFalls JA: Ablehnung der Nebentätigkeit (ggf. Rücksprache mit der Geschäftsführung). \nFalls NEIN: Dokument “Erlaubnis Nebentätigkeit” erstellen. \n\nFalls nötig, kurze Rücksprache mit dem Mitarbeiter oder der Geschäftsführung bei Unklarheiten. \nDokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden. \nUnterschriebenes Dokument einscannen und in der Personalakte ablegen.","Mitarbeiter, HR, Geschäftsführung","Mitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben, Informationen einholen, Prüfen ob die Nebentätigkeit im Wettbewerb zu adorsys steht, Ablehnung der Nebentätigkeit, Dokument “Erlaubnis Nebentätigkeit” erstellen, Rücksprache mit dem Mitarbeiter oder der Geschäftsführung halten, Dokument zur Unterschrift vorlegen und dem Mitarbeiter zusenden, Unterschriebenes Dokument einscannen und in der Personalakte ablegen"
1,"Generiere ein Draw.io-konformes XML-Prozessmodell, das alle bereitgestellten Rollen und Aktivitäten umfasst. Gib das XML-Dokument als Ergebnis zurück.\n'''Process Description: \nMitarbeiter informiert HR über die Absicht, einer Nebentätigkeit nachzugehen oder diese bereits auszuüben (z. B. Selbstständigkeit, Minijob). \nFolgende Informationen einholen: \nArt der Nebentätigkeit (kurze stichpunktartige Beschreibung der Tätigkeiten): XXXXXX \nName des zweiten Arbeitgebers: XXXXXX \nBeginn der Nebentätigkeit: MM/YYYY \nArbeitszeit: max. X Stunden/

In [105]:
#!pip3 install openpyxl

In [ ]:
df.to_excel("assets/dataframe.xlsx")